In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter
import re
import os


In [2]:
# Get the current working directory
cwd = os.getcwd()

# Define the relative path to the CSV file
csv_path = os.path.join(cwd, 'Shitstorms.csv')

# Read the CSV file using pandas
df = pd.read_csv(csv_path, sep=";")

In [3]:
df = df.dropna()
df.head()


,ShitstormID,Profilname,sp1,ep1,sp2,ep2,sp3,ep3,sp4,ep4,sp5,ep5
0,1,barilla,2013-09-24,2013-09-25,2013-09-26,2013-09-26,2013-09-27,2013-09-28,2013-09-29,2013-10-03,2013-10-04,2013-10-10
1,2,otto,2021-10-29,2021-10-30,2021-10-01,2021-11-02,2021-11-03,2021-11-04,2021-11-05,2021-11-10,2021-11-11,2021-11-30
2,3,EMMA_Magazin,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-13,2016-01-14,2016-01-20,2016-01-21,2016-02-10
3,4,Gillette,2019-01-15,2019-01-16,2019-01-17,2019-01-18,2019-01-19,2019-01-21,2019-01-22,2019-01-25,2019-01-26,2019-02-10
4,5,Kaufland,2020-10-07,2020-10-07,2020-10-07,2020-10-07,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-16


In [4]:
# create a list to store the new data
new_data = []

# iterate over the rows of the original dataframe
for index, row in df.iterrows():
    for i in range(1, 6):
        # extract the start and end dates from the original dataframe
        start = row[f'sp{i}']
        end = row[f'ep{i}']
        # create a new row with the desired columns
        new_row = {
            'number': i,
            'start': start,
            'end': end
        }
        # add the new row to the list
        new_data.append(new_row)

# create the new dataframe
new_df = pd.DataFrame(new_data)

# print the new dataframe
print(new_df)

    number       start         end
0        1  2013-09-24  2013-09-25
1        2  2013-09-26  2013-09-26
2        3  2013-09-27  2013-09-28
3        4  2013-09-29  2013-10-03
4        5  2013-10-04  2013-10-10
5        1  2021-10-29  2021-10-30
6        2  2021-10-01  2021-11-02
7        3  2021-11-03  2021-11-04
8        4  2021-11-05  2021-11-10
9        5  2021-11-11  2021-11-30
10       1  2016-01-07  2016-01-08
11       2  2016-01-09  2016-01-10
12       3  2016-01-11  2016-01-13
13       4  2016-01-14  2016-01-20
14       5  2016-01-21  2016-02-10
15       1  2019-01-15  2019-01-16
16       2  2019-01-17  2019-01-18
17       3  2019-01-19  2019-01-21
18       4  2019-01-22  2019-01-25
19       5  2019-01-26  2019-02-10
20       1  2020-10-07  2020-10-07
21       2  2020-10-07  2020-10-07
22       3  2020-10-07  2020-10-08
23       4  2020-10-09  2020-10-10
24       5  2020-10-11  2020-10-16
25       1  2020-01-28  2020-01-28
26       2  2020-01-28  2020-01-28
27       3  2020-01-

In [5]:
for index, row in df.iterrows():
    for j, col in enumerate(row[1:], start=2):
        if j < len(row) - 1:
            query = "(to:{}) until:{} since:{}".format(row['Profilname'],row[j+1], row[j])
            
            print(query)

(to:barilla) until:2013-09-25 since:2013-09-24
(to:barilla) until:2013-09-26 since:2013-09-25
(to:barilla) until:2013-09-26 since:2013-09-26
(to:barilla) until:2013-09-27 since:2013-09-26
(to:barilla) until:2013-09-28 since:2013-09-27
(to:barilla) until:2013-09-29 since:2013-09-28
(to:barilla) until:2013-10-03 since:2013-09-29
(to:barilla) until:2013-10-04 since:2013-10-03
(to:barilla) until:2013-10-10 since:2013-10-04
(to:otto) until:2021-10-30 since:2021-10-29
(to:otto) until:2021-10-01 since:2021-10-30
(to:otto) until:2021-11-02 since:2021-10-01
(to:otto) until:2021-11-03 since:2021-11-02
(to:otto) until:2021-11-04 since:2021-11-03
(to:otto) until:2021-11-05 since:2021-11-04
(to:otto) until:2021-11-10 since:2021-11-05
(to:otto) until:2021-11-11 since:2021-11-10
(to:otto) until:2021-11-30 since:2021-11-11
(to:EMMA_Magazin) until:2016-01-08 since:2016-01-07
(to:EMMA_Magazin) until:2016-01-09 since:2016-01-08
(to:EMMA_Magazin) until:2016-01-10 since:2016-01-09
(to:EMMA_Magazin) until:2

In [6]:
df1 = pd.DataFrame(columns=['ssid', 'phase', 'retweetCount','likeCount','replyCount','content'])
for index, row in df.iterrows():
    
    new_data = []
    for i in range(1, 6):
        # extract the start and end dates from the original dataframe
        start = row[f'sp{i}']
        end = row[f'ep{i}']
        # create a new row with the desired columns
        new_row = {
            'number': i,
            'start': start,
            'end': end
        }
        # add the new row to the list
        new_data.append(new_row)

    # create the new dataframe
    new_df = pd.DataFrame(new_data)
    
    for j, col in new_df.iterrows():
        query = "(to:{}) until:{} since:{}".format(row['Profilname'], col['end'], col['start'])
        limit = 2
        tweets = []

        print (query)
    

(to:barilla) until:2013-09-24 since:2013-09-24
(to:barilla) until:2013-09-25 since:2013-09-25
(to:barilla) until:2013-09-27 since:2013-09-26
(to:barilla) until:2013-10-01 since:2013-09-28
(to:barilla) until:2013-10-15 since:2013-10-02
(to:kitkat) until:2010-03-17 since:2010-03-17
(to:kitkat) until:2010-03-18 since:2010-03-18
(to:kitkat) until:2010-03-19 since:2010-03-19
(to:kitkat) until:2010-03-23 since:2010-03-20
(to:kitkat) until:2010-04-05 since:2010-03-24
(to:otto) until:2021-10-28 since:2021-10-28
(to:otto) until:2021-10-29 since:2021-10-29
(to:otto) until:2021-10-30 since:2021-10-30
(to:otto) until:2021-11-04 since:2021-10-31
(to:otto) until:2021-11-30 since:2021-11-05


In [7]:
df1 = pd.DataFrame(columns=['ssid', 'phase', 'retweetCount','likeCount','replyCount','content'])
for index, row in df.iterrows():
    
    new_data = []
    for i in range(1, 6):
        # extract the start and end dates from the original dataframe
        start = row[f'sp{i}']
        end = row[f'ep{i}']
        # create a new row with the desired columns
        new_row = {
            'number': i,
            'start': start,
            'end': end
        }
        # add the new row to the list
        new_data.append(new_row)

    # create the new dataframe
    new_df = pd.DataFrame(new_data)
    
    for j, col in new_df.iterrows():
        query = "(to:{}) until:{} since:{}".format(row['Profilname'], col['end'], col['start'])
        limit = 5000
        tweets = []

        for tweet in sntwitter.TwitterSearchScraper(query).get_items():
            
            if len(tweets)==limit:
                break
            else:
                
                tweets.append([int(index), int(j), tweet.retweetCount, tweet.likeCount, tweet.replyCount, re.sub(r'@[^\s]+', '', tweet.rawContent, count=1)])

        df1 = df1.append(pd.DataFrame(tweets, columns=['ssid', 'phase', 'retweetCount','likeCount','replyCount','content']))

df1.to_csv('C:/Users/fhb181029/Documents/FH/S9/MasterArbeit/Prototyp/Datascrape.csv', sep=';',index=False)

Stopping after 20 empty pages
C:\Users\fhb181029\AppData\Local\Temp\ipykernel_12504\2455835138.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(pd.DataFrame(tweets, columns=['ssid', 'phase', 'retweetCount','likeCount','replyCount','content']))
Stopping after 20 empty pages
C:\Users\fhb181029\AppData\Local\Temp\ipykernel_12504\2455835138.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(pd.DataFrame(tweets, columns=['ssid', 'phase', 'retweetCount','likeCount','replyCount','content']))
Stopping after 20 empty pages
C:\Users\fhb181029\AppData\Local\Temp\ipykernel_12504\2455835138.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(pd.DataFrame(tweets, columns=['ssid', 

In [5]:
df1.to_csv('C:/Users/fhb181029/Documents/FH/S9/MasterArbeit/Prototyp/Datascrape.csv', sep=';',index=False)

In [8]:
#use this to get by search query
query="(to:arminwolf) until:2023-04-02 since:2023-04-01"

tweets = []
limit = 10

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets)==limit:
        break
    else:
        tweets.append([tweet.inReplyToUser,tweet.rawContent, tweet.url])

df1 = pd.DataFrame(tweets, columns=['inReply','content','url'])

print (df1)

                         inReply  \
0  https://twitter.com/ArminWolf   
1  https://twitter.com/ArminWolf   
2  https://twitter.com/ArminWolf   
3  https://twitter.com/ArminWolf   
4  https://twitter.com/ArminWolf   
5  https://twitter.com/ArminWolf   
6  https://twitter.com/ArminWolf   
7  https://twitter.com/ArminWolf   
8  https://twitter.com/ArminWolf   
9  https://twitter.com/ArminWolf   

                                             content  \
0  @ArminWolf Mißratener Aprilscherz, oder hat er...   
1  @ArminWolf Fakt ist, dass an den Stammtischen ...   
2  @ArminWolf Sehe ich ganz genauso.\nAlternative...   
3  @ArminWolf Danke, Herr Wolf für ihre objektive...   
4               @ArminWolf Safe, oder? @SWagenknecht   
5  @ArminWolf Ja ab und zu schon. Zwar nicht imme...   
6  @ArminWolf Wenn die“ Banker" zuerst etwa 8 Jah...   
7  @ArminWolf Der ach-so-tolle-Journalist sollte ...   
8  @ArminWolf Immer dran denken: wer andere zwing...   
9  @ArminWolf Einen schurkenstaat wir die U

In [7]:
for i, tweet in enumerate(scraper.get_items()):
    if i>10:
        break
    print (tweet.inReplyToUser,tweet.rawContent, tweet.url)


https://twitter.com/alfabet1999 @alfabet1999 Ja. https://twitter.com/ArminWolf/status/1647934211498688515
https://twitter.com/yhigh @yhigh @FHabersberger Wenn Sie mir ein Finanzprodukt zeigen, dass mit einer Kapitalgarantie (!) seit 2003 im Schnitt 6% p.a. gemacht hat, nehme ich alles zurück. https://twitter.com/ArminWolf/status/1647933896665833477
https://twitter.com/Sprenger3Kl @Sprenger3Kl @elflanell @FranzZapflhuber Sie meinen, dass ich vor 40 Jahren Mitglied einer polit. Jugendorganisation war, aus der ich ausgetreten bin, als ich Journalist wurde, widerspricht „dezitiert“ meinem objektiven Journalismus? Ah ja. 
Mein Tipp: Don‘t drink and tweet! 
Salut! https://twitter.com/ArminWolf/status/1647933081053978625
https://twitter.com/MichaelCsoklich @MichaelCsoklich Klar, aber auch beim erwarteten Zinssatz waren die 6% auf Dauer immer irreal. Es gab und gibt historisch keine Veranlagung mit Kapitalgarantie, die dauerhaft 6% jährlich bringt. https://twitter.com/ArminWolf/status/16479031

In [57]:
# get replys 

mode = sntwitter.TwitterTweetScraperMode
sncraper_reply = sntwitter.TwitterTweetScraper(tweetId=1647894575510372352, mode=mode.SCROLL)

for i, tweet in enumerate(sncraper_reply.get_items()):
    if i>5:
        break
    print (tweet.url)


https://twitter.com/ArminWolf/status/1647894575510372352
https://twitter.com/ArminWolf/status/1647895245575651329
https://twitter.com/emmaKnigthly/status/1647899455960629249
https://twitter.com/FunkhausZorra/status/1647900187963039744
https://twitter.com/veronikaDapra/status/1647978166604931073
https://twitter.com/veronikaDapra/status/1647978914638118914


In [60]:
mode = sntwitter.TwitterTweetScraperMode
sncraper_reply = sntwitter.TwitterTweetScraper(tweetId=1647894575510372352, mode=mode.SCROLL)

print (sncraper_reply.get_items(tweet.content))

replies = list(scraper.get_items())
number_of_replies = sum(1 for _ in replies)

for tweet in replies:
    print(tweet.content)

C:\Users\fhb181029\AppData\Local\Temp\ipykernel_13076\1214727230.py:4: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  print (sncraper_reply.get_items(tweet.content))


TypeError: get_items() takes 1 positional argument but 2 were given